In [24]:
from bs4 import BeautifulSoup
import requests as req
import pandas as pd
import re


## Get map list from game res and map type from wotreplays.ru

In [3]:
with open("D:\\Games\\World_of_Tanks\\res\\text\lc_messages\\arenas.mo", "rb") as f:
    arenas = f.read()


In [20]:
work_names_start = arenas.find(b"01_karelia")
work_names_end = arenas.find(b"Project-Id-Version: World Of Tanks") - 1

desc_start = arenas.find(b"Content-Transfer-Encoding: 8bit\n") + \
    len("Content-Transfer-Encoding: 8bit\n") + 1
desc_end= len(arenas) - 1


In [21]:
work_names = [i.decode("utf8") for i in arenas[work_names_start:work_names_end].split(b"\x00")]
desc = [i.decode("utf8") for i in arenas[desc_start:desc_end].split(b"\x00")]


In [45]:
maps = []
for wn, d in zip(work_names, desc):
    if re.match(r"^\d+_.*name$", wn):
        id_map = int(wn.split("_")[0])
        is_standart = len(BeautifulSoup(req.get(
            f"http://wotreplays.ru/site/index/battle_type/1/map/{id_map}").text, "html.parser") \
            .find_all(class_="btn_l-grey")[:-1]) != 0
        maps.append({
            "id": id_map,
            "work_name": wn[:-5],
            "name": d,
            "standart": is_standart
        })
maps = pd.DataFrame(maps)

In [50]:
maps.to_csv("../data/assets/maps.csv", index=False)